# Simplex tableau algorithm: phase I

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import numpy as np
from teaching_optimization.simplex_tableau_phase_two import (
    SimplexTableauPhaseTwo,
)
from teaching_optimization.tableau import SimplexTableau, RowColumn


We want to solve three optimization problems using the simplex algorithm.

First, Example 16.15, p. 388 of
[Bierlaire (2015)](https://transp-or.epfl.ch/books/optimization/html/OptimizationPrinciplesAlgorithms2018.pdf)

$$\min_{x \in \mathbb{R}^4} x_1 + x_2 + x_3 $$
subject to
$$ \begin{align*}
x_1 + 2x_2 + 3 x_3 &= 3, \\
-x_1 + 2 x_2 + 6 x_3  &= 2, \\
4 x_2 + 9 x_3 &= 5, \\
3 x_3 + x_4 &= 1, \\
x_1, x_2, x_3, x_4 & \geq 0.
\end{align*} $$

Data for the standard form:

In [ ]:
objective_1 = np.array([1, 1, 1, 0])
constraints_1 = np.array(
    [[1, 2, 3, 0], [-1, 2, 6, 0], [0, 4, 9, 0], [0, 0, 3, 1]]
)
rhs_1 = np.array([3, 2, 5, 1])


Second, Example 16.15, .p. 392 of
[Bierlaire (2015)](https://transp-or.epfl.ch/books/optimization/html/OptimizationPrinciplesAlgorithms2018.pdf)

$$\min_{x \in \mathbb{R}^5} 2 x_1 + 3 x_2 + 3 x_3 + x_4 - 2 x_5 $$
subject to
$$ \begin{align*}
-x_1 - 3x_2 - 4 x_4 - x_5 &= -2, \\
x_1 + 2 x_2 - 3 x_4 + x_5 &= 2, \\
-x_1 -4 x_2 + 3 x_3 &= 1, \\
x_1, x_2, x_3, x_4, x_5 & \geq 0.
\end{align*} $$

Data for the standard form:

In [ ]:
objective_2 = np.array([2, 3, 3, 1, -2])
constraints_2 = np.array(
    [[-1, -3, 0, -4, -1], [1, 2, 0, -3, 1], [-1, -4, 3, 0, 0]]
)
rhs_2 = np.array([-2, 2, 1])


Finally, we will try to solve the following infeasible problem:
$$ \min_{x_1 \in \mathbb{R}}$$
subject to
$$ \begin{align*}
x_1 & \leq -1, \\
x_1 & \geq 0.
\end{align*} $$

Data in standard form:

In [ ]:
objective_3 = np.array([1, 0])
constraints_3 = np.array([1, 1])
rhs_3 = np.array([-1])



First, we need to prepare the initial tableau for Phase I:

- By multiplying the relevant constraints by $-1$, modify the problem
such that $b \geq 0$.

- Introduce the auxiliary variables $x^a_1,\ldots, x^a_m$ and
define $T_0$ as
$$
\begin{array}{|c|c|c|}
\hline
x_1  \ldots  x_n & x^a_1  \ldots  x^a_m & \\
\hline
A & I & b \\
\hline
-e^TA & 0 & -e^Tb\\
\hline
\end{array}
$$
where $e$ is the vector of $\mathbb{R}^m$ for which all components are $1$.

Replace the ???? by valid code.

In [ ]:


def initial_tableau_phase_one(
    constraint_matrix: np.ndarray, right_hand_side: np.ndarray
) -> SimplexTableau:
    """Generates the initial tableau for phase I

    :param constraint_matrix: matrix containing the constraints coefficients.
    :param right_hand_side: right hand side of the constraints.
    :return: the first tableau for Phase I
    """
    # We first make sure that the matrix is indeed stored as a matrix
    constraint_matrix = np.atleast_2d(constraint_matrix)
    n_constraints, n_variables = constraint_matrix.shape

    # We need to make sure that all entries of the right hand side are non negative.
    # Identify first the negative entries in the right hand side
    negative_mask = right_hand_side < 0
    # Then, if needed, multiply the corresponding entries and the corresponding constraints by -1.
    constraint_matrix[negative_mask] *= -1
    right_hand_side[negative_mask] *= -1

    # We now build the tableau by define each component, and stacking them horizontally, then vertically.
    # First row of components
    left = constraint_matrix
    middle = np.eye(n_constraints)  # Identity matrix of size m
    # To be used with hstack, we need to add a new axis to the right hand side, to make it a matrix instead of a vector.
    right = right_hand_side[:, np.newaxis]
    first_row = np.hstack((left, middle, right))
    # Second row components
    e = np.ones(n_constraints)  # Vector of ones of size m
    left = -np.dot(e, constraint_matrix)  # -e^T A
    middle = np.zeros(n_constraints)  # m zeros
    right = -np.dot(e, right_hand_side)  # -e^T b

    second_row = np.hstack(
        (
            left,
            middle,
            right,
        )
    )

    # Combine both rows
    complete_tableau = np.vstack((first_row, second_row))
    the_tableau = SimplexTableau(tableau=complete_tableau)
    return the_tableau



We test the function with the first example

In [ ]:
initial_tableau_1 = initial_tableau_phase_one(
    constraint_matrix=constraints_1, right_hand_side=rhs_1
)


Expected result:
```
[[ 1.   2.   3.   0.   1.   0.   0.   0.   3.]
[ -1.   2.   6.   0.   0.   1.   0.   0.   2.]
[  0.   4.   9.   0.   0.   0.   1.   0.   5.]
[  0.   0.   3.   1.   0.   0.   0.   1.   1.]
[ -0.  -8. -21.  -1.   0.   0.   0.   0. -11.]]
```

In [ ]:
print(initial_tableau_1)



We now solve the auxiliary problem using the simplex algorithm.
Add the condition that verifies if the problem is infeasible. In that case, None is returned.

In [ ]:
def solve_phase_one(initial_tableau: SimplexTableau) -> SimplexTableau | None:
    """Solve the auxiliary problem, and clean the optimal tableau. If the original problem is not feasible,
    return None"""

    algorithm = SimplexTableauPhaseTwo(initial_tableau=initial_tableau)

    optimal_tableau_phase_one = initial_tableau
    for tableau in algorithm:
        optimal_tableau_phase_one = tableau

    if not np.isclose(
        optimal_tableau_phase_one.value_objective_function, 0.0
    ):
        return None
    return optimal_tableau_phase_one



We test the function with the first example

In [ ]:
optimal_tableau_phase_one_1 = solve_phase_one(initial_tableau=initial_tableau_1)


Expected result
```
[[    1       0       0     0.5     0.5    -0.5       0     0.5       1]
[     0       1       0   -0.75    0.25    0.25       0   -0.75     0.5]
[     0       0       0       0      -1      -1       1       0       0]
[     0       0       1   0.333       0       0       0   0.333   0.333]
[     0       0       0       0       2       2       0       1       0]]
```

In [ ]:
print(optimal_tableau_phase_one_1)



We now "clean" the tableau before starting Phase II. It consists in

- moving auxiliary variables out of the basis,
- removing redundant constraints,

In [ ]:
def remove_auxiliary_variables_from_basis(
    optimal_tableau_phase_one: SimplexTableau,
) -> SimplexTableau:
    """Remove auxiliary variables and possibly redundant constraints from the optimal tableau"""

    # We deduce from the number of variables in the tableau the number of variables in the original problem.
    n_constraints = optimal_tableau_phase_one.n_constraints
    n_total_variables = optimal_tableau_phase_one.n_variables
    n_original_variables = n_total_variables - n_constraints

    clean_tableau = False
    while not clean_tableau:
        basic_variables = optimal_tableau_phase_one.identify_basic_variables()
        for basic_var in basic_variables:
            # You can access to the index of the row and the index of the column using
            # `basic_var.row` and `basic_var.column`.

            # Check if the basic variable is auxiliary.
            if (
                basic_var.column >= n_original_variables
            ) and (
                basic_var.column < (n_original_variables + n_constraints)
            ):
                # Find the non zero elements in the corresponding part of the row, corresponding to original variables.
                # First, define a "slice", that identifies elements of the row corresponding to original variables.
                row_slice = optimal_tableau_phase_one.tableau[
                    basic_var.row, :n_original_variables
                ]
                # Then, identify all non zero elements in that slice
                nonzero_indices = (
                    np.nonzero(row_slice)[0]
                )

                if len(nonzero_indices) == 0:
                    """No way to pivot. The constraint is redundant and must be removed"""
                    optimal_tableau_phase_one = optimal_tableau_phase_one.remove_row(
                        basic_var.row
                    )
                    break

                # We pivot to remove the auxiliary variable out of the basis
                pivot = RowColumn(row=basic_var.row, column=int(nonzero_indices[0]))
                optimal_tableau_phase_one.pivoting(pivot=pivot)
                break

        else:
            clean_tableau = True

    return optimal_tableau_phase_one



We test the function with the first example

In [ ]:
pivoted_tableau_1 = remove_auxiliary_variables_from_basis(
    optimal_tableau_phase_one=optimal_tableau_phase_one_1
)


Expected result:
```
[[    1       0       0     0.5     0.5    -0.5       0     0.5       1]
[     0       1       0   -0.75    0.25    0.25       0   -0.75     0.5]
[     0       0       1   0.333       0       0       0   0.333   0.333]
[     0       0       0       0       2       2       0       1       0]]
```

In [ ]:
print(pivoted_tableau_1)



We can now prepare the tableau for Phase two. It involves:

- remove columns corresponding to auxiliary variables, and,
- calculate the reduced costs.


In [ ]:
def prepare_tableau_for_phase_two(
    cost_vector: np.ndarray, cleaned_tableau: SimplexTableau
) -> SimplexTableau:
    """Remove columns corresponding to auxiliary variables and calculate the reduced costs.

    :param cost_vector: cost coefficients of the original problem
    :param cleaned_tableau: tableau where all auxiliary variables have been removed from the basis
    :return: initial tableau for phase II
    """
    # Note that, after cleaning, the number of constraints may not correspond anymore to the number of auxialiary
    # variables, as redundant constraints have been re moved.
    n_total_variables = optimal_tableau_phase_one_1.n_variables
    n_original_variables = len(cost_vector)

    basic_variables = cleaned_tableau.identify_basic_variables()
    for basic_var in basic_variables:
        if basic_var.column >= n_original_variables and (
            basic_var.column < n_total_variables
        ):
            error_msg = f'Auxiliary variable {basic_var.column} is in the basis. Tableau cannot be cleaned'
            raise ValueError(error_msg)

    # Delete the columns corresponding to the auxiliary variables.
    # Remember that a slice includes the first element, but not the last
    slice_to_delete = np.s_[
        n_original_variables:n_total_variables
    ]
    new_array = np.delete(cleaned_tableau.tableau, slice_to_delete, axis=1)
    phase_two_tableau = SimplexTableau(tableau=new_array)

    # Calculate the reduced costs
    phase_two_tableau.recalculate_reduced_costs(costs=cost_vector)

    return phase_two_tableau



We test the function with the first example

In [ ]:
initial_tableau_phase_two_1 = prepare_tableau_for_phase_two(
    cost_vector=objective_1, cleaned_tableau=pivoted_tableau_1
)


Expected result:
```
[[    1       0       0     0.5       1]
[     0       1       0   -0.75     0.5]
[     0       0       1   0.333   0.333]
[     0       0       0  -0.0833   -1.83]]
```

In [ ]:
print(initial_tableau_phase_two_1)


Now that we have an initial tableau, we can solve Phase two.

In [ ]:
phase_two_1 = SimplexTableauPhaseTwo(initial_tableau=initial_tableau_phase_two_1)

optimal_tableau_1 = None
for tableau in phase_two_1:
    optimal_tableau_1 = tableau


The reason why the algorithm stopped can be checked.

In [ ]:
stopping_cause = phase_two_1.stopping_cause
print(stopping_cause)


The optimal solution is

In [ ]:
print(phase_two_1.solution)


Expected optimal tableau:
```
[[    1       0    -1.5       0     0.5]
[     0       1    2.25       0    1.25]
[     0       0       3       1       1]
[     0       0    0.25       0   -1.75]]
```

In [ ]:
print(optimal_tableau_1)



# Second example.
We now apply the same procedure to the second example.
There is no need to code anymore. But verify that the output is as expected.

Initial tableau for phase one.

In [ ]:
initial_tableau_2 = initial_tableau_phase_one(
    constraint_matrix=constraints_2, right_hand_side=rhs_2
)


Expected result:
```
[[    1       3       0       4       1       1       0       0       2]
[     1       2       0      -3       1       0       1       0       2]
[    -1      -4       3       0       0       0       0       1       1]
[    -1      -1      -3      -1      -2       0       0       0      -5]]
```

In [ ]:
print(initial_tableau_2)


Solving phase one.

In [ ]:
optimal_tableau_phase_one_2 = solve_phase_one(initial_tableau=initial_tableau_2)


Expected result:
```
[[    1       3       0       4       1       1       0       0       2]
[     0      -1       0      -7       0      -1       1       0       0]
[     0  -0.333       1    1.33   0.333   0.333       0   0.333       1]
[     0       1       0       7       0       2       0       1       0]]
```

In [ ]:
print(optimal_tableau_phase_one_2)


Remove auxiliary variables from the basis.

In [ ]:
pivoted_tableau_2 = remove_auxiliary_variables_from_basis(
    optimal_tableau_phase_one=optimal_tableau_phase_one_2
)


Expected result:
```
[[    1       0       0     -17       1      -2       3       0       2]
[    -0       1      -0       7      -0       1      -1      -0      -0]
[     0       0       1    3.67   0.333   0.667  -0.333   0.333       1]
[     0       0       0       0       0       1       1       1       0]]
```

In [ ]:
print(pivoted_tableau_2)


Prepare the tableau for phase two.

In [ ]:
initial_tableau_phase_two_2 = prepare_tableau_for_phase_two(
    cost_vector=objective_2, cleaned_tableau=pivoted_tableau_2
)


Expected result:
```
[[    1       0       0     -17       1       2]
[    -0       1      -0       7      -0      -0]
[     0       0       1    3.67   0.333       1]
[     0       0       0       3      -5      -7]]
```

And, finally, we solve phase two.

In [ ]:
phase_two_2 = SimplexTableauPhaseTwo(initial_tableau=initial_tableau_phase_two_2)

optimal_tableau_2 = None
for tableau in phase_two_2:
    optimal_tableau_2 = tableau


The reason why the algorithm stopped can be checked.

In [ ]:
print(phase_two_2.stopping_cause)


The optimal solution is

In [ ]:
print(phase_two_2.solution)


Expected optimal tableau:
```
[[     1    2.43       0       0       1       2]
[     0   0.143       0       1       0       0]
[-0.333   -1.33       1       0       0   0.333]
[     5    11.7       0       0       0       3]]
```

In [ ]:
print(optimal_tableau_2)


# Third example.
We now apply the same procedure to the third example, which is an infeasible problem.
There is no need to code anymore. But verify that the output is as expected.

Initial tableau for phase one.

In [ ]:
initial_tableau_3 = initial_tableau_phase_one(
    constraint_matrix=constraints_3, right_hand_side=rhs_3
)


Expected result:
```
[[   -1      -1       1       1]
[     1       1       0      -1]]
```

Solving phase one.

In [ ]:
optimal_tableau_phase_one_3 = solve_phase_one(initial_tableau=initial_tableau_3)


As the problem is infeasible, the function must return None

In [ ]:
print(optimal_tableau_phase_one_3)